<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M1-recommender-nomadlis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [306]:
import pandas as pd

In [307]:
trips = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/trips.csv')

In [308]:
trips['date_end'] = pd.to_datetime(trips.date_end, errors='coerce')
trips['date_start'] = pd.to_datetime(trips.date_start, errors='coerce')

In [309]:
trips['date_start'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                   46499
unique                   4497
top       2017-05-01 00:00:00
freq                       90
first     1970-01-01 00:00:00
last      2106-06-19 00:00:00
Name: date_start, dtype: object

In [310]:
trips['date_end'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                   46292
unique                   4478
top       2016-06-30 00:00:00
freq                       74
first     1970-01-01 00:00:00
last      2222-01-01 00:00:00
Name: date_end, dtype: object

In [311]:
first = trips['date_start'].quantile(0.05)

In [312]:
last = trips['date_end'].quantile(0.95)

In [313]:
trips = trips[(trips.date_start >= first) & (trips.date_end <= last)]

In [314]:
import scipy.sparse as ss
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [315]:
le_user = LabelEncoder()
le_place = LabelEncoder()

In [316]:
trips['username_id'] = le_user.fit_transform(trips['username'])

In [317]:
trips['place_slug_id'] = le_place.fit_transform(trips['place_slug'])

In [318]:
ones = np.ones(len(trips), np.uint32)

In [319]:
matrix = ss.coo_matrix((ones, (trips['username_id'], trips['place_slug_id'])))

In [59]:
matrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [320]:
np.where(matrix.todense()[1] == 1)

(array([0, 0]), array([433, 792]))

In [58]:
trips[trips['username_id'] == 1]

,Unnamed: 0,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug,username_id,place_slug_id
699,699,@0chucha0,Sri Lanka,LK,sri-lanka,2018-03-18,2018-02-18,7,80,Sri Lanka,sri-lanka,1,792
700,700,@0chucha0,Thailand,TH,thailand,2018-02-20,2017-11-21,9,100,Ko Samui,ko-samui-thailand,1,433


In [321]:
from sklearn.decomposition import TruncatedSVD

In [322]:
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

In [323]:
matrix_users = svd.fit_transform(matrix)

In [324]:
matrix_places = svd.fit_transform(matrix.T)

In [325]:
matrix_places

array([[ 1.81576739e-02, -2.55111605e-03,  6.15248871e-02,
        -1.82531859e-02, -1.30631662e-02],
       [ 6.43184382e-02, -2.56848567e-03,  7.82669077e-02,
         4.33770282e-01, -3.43629704e-01],
       [ 2.22883998e-02,  6.08413036e-04,  9.33975454e-03,
         4.74407091e-02,  2.20192149e-02],
       ...,
       [ 3.80432584e+00, -5.29492371e-02,  1.71337042e+00,
         1.25019246e+00, -4.49876564e-01],
       [ 3.92807938e-01,  8.41059925e-01, -1.38472884e-01,
        -2.99649996e-02, -3.31083026e-02],
       [ 4.98476098e-01,  4.06888171e-03,  2.86557464e-01,
         3.71966324e-01,  4.92203378e-01]])

In [326]:
# or easier
from sklearn.metrics.pairwise import cosine_distances

In [327]:
cosine_distance_matrix_places = cosine_distances(matrix_places)

In [328]:
cosine_distance_matrix_places.shape

(927, 927)

In [330]:
le_place.transform(['ubud-bali-indonesia'])

array([858])

In [360]:
def similar_place(place, n):
  ix = le_place.transform([place])[0]
  sim_places = le_place.inverse_transform(np.argsort(cosine_distance_matrix_places[ix,:])[:n])
  return sim_places

In [359]:
similar_place('ko-pha-ngan-thailand', 10)

array(['ko-pha-ngan-thailand', 'ubud-bali-indonesia', 'pai-thailand',
       'hoi-an-vietnam', 'yogyakarta-indonesia', 'laos',
       'ho-chi-minh-city-vietnam', 'malaysia', 'vietnam',
       'canberra-australia'], dtype=object)

In [331]:
np.argsort(cosine_distance_matrix_places[858,:])[:5]

array([858, 431, 914, 347, 513])

In [332]:
le_place.inverse_transform(np.argsort(cosine_distance_matrix_places[858,:])[:10])

array(['ubud-bali-indonesia', 'ko-pha-ngan-thailand',
       'yogyakarta-indonesia', 'ho-chi-minh-city-vietnam', 'malaysia',
       'gurgaon-india', 'laos', 'siem-reap-cambodia', 'lviv-ukraine',
       'kathmandu-nepal'], dtype=object)

In [336]:
trips[trips.username_id == 7]

,Unnamed: 0,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug,username_id,place_slug_id
44611,44611,@9102180,Indonesia,ID,indonesia,2018-01-31,2018-01-14,-8,115,Canggu,canggu-bali-indonesia,7,159
44612,44612,@9102180,Thailand,TH,thailand,2018-01-13,2017-12-01,13,100,Bangkok,bangkok-thailand,7,75
44613,44613,@9102180,Thailand,TH,thailand,2017-09-30,2017-08-14,18,98,Chiang Mai,chiang-mai-thailand,7,180


In [338]:
user_places_ids = trips[trips.username_id == 7]['place_slug_id'].unique()
user_vector_trips = np.mean(matrix_places[user_places_ids], axis=0)

In [339]:
closest_for_user = cosine_distances(user_vector_trips.reshape(1,5), matrix_places)

In [340]:
le_place.inverse_transform(np.argsort(closest_for_user[0])[:10])

array(['jakarta-indonesia', 'phnom-penh-cambodia', 'darwin-australia',
       'shenyang-china', 'guiyang-china', 'north-korea',
       'sinuiju-north-korea', 'chongqing-china', 'bangkok-thailand',
       'siem-reap-cambodia'], dtype=object)

In [352]:
def similar_user_place(u_id, n):
  user_places_ids = trips[trips.username_id == u_id]['place_slug_id'].unique()
  user_vector_trips = np.mean(matrix_places[user_places_ids], axis=0)
  closest_for_user = cosine_distances(user_vector_trips.reshape(1,5), matrix_places)
  sim_places = le_place.inverse_transform(np.argsort(closest_for_user[0])[:n])
  return sim_places

In [374]:
def similar_multiplaces(places, n):
  ixs = le_place.transform(places)
  places_vector = np.mean(matrix_places[ixs], axis=0)
  closest_multiple = cosine_distances(places_vector.reshape(1,5), matrix_places)
  sim_places = le_place.inverse_transform(np.argsort(closest_multiple[0])[:n])
  return sim_places

In [378]:
similar_multiplaces(['lviv-ukraine','darwin-australia'], 20)

array(['lviv-ukraine', 'yogyakarta-indonesia', 'ho-chi-minh-city-vietnam',
       'siem-reap-cambodia', 'jakarta-indonesia', 'malaysia',
       'kuala-lumpur-malaysia', 'phnom-penh-cambodia', 'darwin-australia',
       'shenyang-china', 'north-korea', 'sinuiju-north-korea',
       'guiyang-china', 'laos', 'kunming-china', 'chongqing-china',
       'chiang-mai-thailand', 'windsor-canada', 'yokohama-japan',
       'canberra-australia'], dtype=object)